In [5]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import xlsxwriter
import datetime

In [6]:
# ususaarios de base de datos local

usuario = "care_consulta"
contrasena = "CareAssistance!10"
url_servidor = "10.0.10.22"
# puerto = '3306'
# esquema = 'EspoCRM2'
# plugin_autenticacion = 'mysql_native_password'

# usuarios de base de datos Simtastic ##########################################

#usuario = 'siempreread'
#contrasena = 'S13mpr3_Ch1l3'
#url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "EspoCRM2"
plugin_autenticacion = "mysql_native_password"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

In [7]:
sql = "select cas.number as 'Id',trim(con.rut) as 'Rut Paciente',trim(con.rut_asegurado) as 'Rut Titular',con.relacion_con_asegurado as 'Relación con asegurado', cas.area as 'Prestacion',"
sql2 = " cas.created_at as 'Fecha Gestión', con.empleador AS 'Empleador', cas.channel as 'Canal', trim(con.campana_paciente) as 'Campaña', con.holding as 'holding'"
sql3 = "from EspoCRM2.case cas left join EspoCRM2.contact con on cas.contact_id = con.id where cas.created_at > '2022-10-01' and cas.deleted = 0 and cas.status <> 'Cancelled' order by cas.number asc ;"

# archivo de salida modificar segun archivo
archivo = "C:/Users/jmartinez/Desktop/Informes/mexico.xlsx"

fin_sql = sql + sql2 + sql3

In [8]:
conchaytoro = pd.read_sql_query(fin_sql, motor_mysql_mariadb)



conchaytoro.head(10)


,Id,Rut Paciente,Rut Titular,Relación con asegurado,Prestacion,Fecha Gestión,Empleador,Canal,Campaña,holding
0,249133,15685631-2,15685631-2,AS,Siempre - Medicina,2022-10-01 00:28:16,Asesorias Otarola Y Fuentes Spa,Video Call,Metlife,None
1,249134,19396110-K,19396110-K,AS,Siempre - Psicología,2022-10-01 00:30:00,komatsu,Video Call,Metlife,None
2,249135,18538089-0,18538089-0,AS,Siempre - Psicología,2022-10-01 00:36:40,Comercial Kaufmann S.A,Video Call,KAUFMANN,None
3,249136,22395450-2,13878914-4,HI,Siempre - Medicina,2022-10-01 03:33:54,Ingenieria Y Montaje Cpr Ltda,Video Call,Metlife,None
4,249137,19398598-K,12349721-K,HI,Siempre - Clínica del Sueño,2022-10-01 11:55:33,Rendic Hnos. S.A.,Llamada telefónica,Metlife,None
5,249138,28123756-9,28123756-9,AS,Siempre - Medicina,2022-10-01 11:58:05,TECREIN SPA,Video Call,Prueba,None
6,249139,16215692-6,14104666-7,CO,Siempre - Medicina,2022-10-01 12:05:43,komatsu,Video Call,Metlife,None
7,249140,LOPE43485424JFE345,LOPE43485424JFE345,AS,Siempre - Medicina,2022-10-01 12:07:01,TECREIN SPA,Video Call,COLABORADORES CARE MX,None
8,249141,14356980-2,14356980-2,AS,Siempre - Psicología,2022-10-01 12:17:13,Agrosuper,Llamada telefónica,Metlife,None
9,249142,16215692-6,14104666-7,CO,Siempre - Medicina,2022-10-01 12:24:55,komatsu,Llamada telefónica,Metlife,None


In [9]:
conchaytoro.shape

(21018, 10)

In [10]:
relacion = {
    "HI": "Hijo/a",
    "AS": "Titular",
    "CO": "Cónyuge",
    "P": "Padre",
    "O": "Otros",
    "M": "Madre",
    "H": "Hijo/a",
    "NI":"Nieto/a"
}
conchaytoro.replace({"Relación con asegurado": relacion}, inplace=True)

prestaciones = {
    "Deporte": "Programa Deportivo",
    "Nutrición": "Programa Nutricional",
    #"Pcte. Crónico": "Programa Enfermería",
    "Orientación": "Orientación Hospitalaria",
    "Psicología": "Programa de Psicología",
    "Psicología": "Programa de Psicología",
    "Medicina": "Doctor Online",
    "Deporte": "Programa Deportivo",
    "Pcte. Crónico": "Programa de Hospitalización Domiciliaria",
    "Pcte. Crónico": "Programa de Hospitalización Domiciliaria",
    "Medicina": "Doctor Online",
    "Psicología": "Programa de Psicología",
    "Nutrición": "Programa Nutricional",
    "Deporte": "Programa Deportivo",
    "Fidelización": "Fidelización",
    "Orientación": "Orientación Hospitalaria",
    #"Pcte. Crónico": "Programa Enfermería",
    "Medicina": "Doctor Online",
    "Agenda": "Orientación Hospitalaria",
    "Fidelización y SAC": "Orientación Hospitalaria",
    "Ingreso dep.  seguro": "Programa Deportivo",
    #"Clínica del Sueño": "Programa Enfermería",
    "Ingreso dep. seguro": "Programa Deportivo",
    "APSSex": "Programa de Psicología",
    "Kinesiología HC": "Programa Deportivo",
    "Siempre - Medicina": "Doctor Online",
    "Siempre - Orientación": "Orientación Hospitalaria",
    "Siempre - Agenda": "Orientación Hospitalaria",
    "Siempre - Deportiva": "Programa Deportivo",
    "Siempre - Psicología": "Programa de Psicología",
    "Siempre - Nutrición": "Programa Nutricional",
    "Siempre  - Front": "Orientación Hospitalaria",
    "Siempre - Clínica del Sueño": "Clinica del Sueño",
    #"Siempre - Enfermería": "Programa Enfermería",
    "Telemedicina": "Doctor Online",
    "Nutricion": "Programa Nutricional",
    "Front": "Orientación Hospitalaria",
    "Deportivo": "Programa Deportivo",
}
#conchaytoro.replace({"Prestacion": prestaciones}, inplace=True)


conchaytoro["derty"] = pd.to_datetime(
    conchaytoro["Fecha Gestión"], format="dd-md-yyyy h:mm"
)
conchaytoro.derty = conchaytoro.derty.dt.strftime("%d-%m-%Y %H:%M")
# conchaytoro['Fecha Gestión'] = conchaytoro['Fecha Gestión'].apply(lambda _: datetime.strptime(_,"%d-%m-%Y, %H:%M"))

In [11]:
conchaytoro.head(50)

,Id,Rut Paciente,Rut Titular,Relación con asegurado,Prestacion,Fecha Gestión,Empleador,Canal,Campaña,holding,derty
0,249133,15685631-2,15685631-2,Titular,Siempre - Medicina,2022-10-01 00:28:16,Asesorias Otarola Y Fuentes Spa,Video Call,Metlife,None,01-10-2022 00:28
1,249134,19396110-K,19396110-K,Titular,Siempre - Psicología,2022-10-01 00:30:00,komatsu,Video Call,Metlife,None,01-10-2022 00:30
2,249135,18538089-0,18538089-0,Titular,Siempre - Psicología,2022-10-01 00:36:40,Comercial Kaufmann S.A,Video Call,KAUFMANN,None,01-10-2022 00:36
3,249136,22395450-2,13878914-4,Hijo/a,Siempre - Medicina,2022-10-01 03:33:54,Ingenieria Y Montaje Cpr Ltda,Video Call,Metlife,None,01-10-2022 03:33
4,249137,19398598-K,12349721-K,Hijo/a,Siempre - Clínica del Sueño,2022-10-01 11:55:33,Rendic Hnos. S.A.,Llamada telefónica,Metlife,None,01-10-2022 11:55
...,...,...,...,...,...,...,...,...,...,...,...
45,249178,10465257-3,10465257-3,Titular,Siempre - Enfermería,2022-10-01 13:42:38,clalemana,ACCUHEALTH,METLIFE,None,01-10-2022 13:42
46,249179,17019451-9,17019451-9,Titular,Siempre - Psicología,2022-10-01 13:44:16,Bridgestone Mining Solutions LA S.A.,Llamada telefónica,Metlife,None,01-10-2022 13:44
47,249180,10465257-3,10465257-3,Titular,Siempre - Enfermería,2022-10-01 13:44:31,clalemana,ACCUHEALTH,METLIFE,None,01-10-2022 13:44
48,249181,10465257-3,10465257-3,Titular,Siempre - Enfermería,2022-10-01 13:45:13,clalemana,ACCUHEALTH,METLIFE,None,01-10-2022 13:45


In [12]:
conchaytoro.to_excel(archivo, index=False)